# Vizard Template Notebook

A minimal template for creating visualizations with Vizard.

**Quick Reference:**
- `%cc KEYWORDS` - Show current state
- `%cc RESET` - Clear state, start fresh
- `%cc HELP` - Show help

**Basic syntax:**
```
%cc DATA mydata.csv PLOT bar X category Y value
%cc Create a scatter plot from mydata.csv with X col1 and Y col2
```

## Setup & Basics

### Load necessary imports (one-time setup)
Since Vizard defaults to IMPORT false, we load libraries once here.

In [2]:
import os, sys, subprocess, warnings, time, re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path

## Altair

In [3]:
alt.renderers.enable('html')
#alt.renderers.enable('default') # Critical

fontFamily = '' # Seems better to leave empty and go with defaults
@alt.theme.register('bioinformatics_theme', enable=True)
def bioinformatics_theme():
    return alt.theme.ThemeConfig({
        'width':  600,
        'height': 400, 
        'config': {
            # Separate axis configurations
            'axisX': {
                'labelFontSize': 14,
                'titleFontSize': 18,
                'titleFontWeight': 100,
                'labelAngle': 0,
                'labelFont': fontFamily,
                'titleFont': fontFamily,  
            },
            'axisY': {
                'labelFontSize': 14,
                'titleFontSize': 18,
                'titleFontWeight': 100,
                'labelAngle': 0,
                'labelFont': fontFamily,
                'titleFont': fontFamily,  
            },
            # Title configuration
            'title': {                
                'fontSize': 25,       
                'fontWeight': 200,    
                'offset': 20,
                'font': fontFamily,        
            },
            # Legend configuration
            'legend': {
                'labelFontSize': 14,
                'titleFontSize': 18,
                'symbolSize': 600,
                'titleFontWeight': 200,
                'labelFont': fontFamily,
                'titleFont': fontFamily,   
            },
            # Text mark configuration
            'text': {
                'font': fontFamily,
                'fontWeight': 100,
                #'fontSize': 14             # Optional: set default size
            }
        }})

#### Fonts

In [4]:
fonts_to_test = [
    # Most reliable cross-platform
    'Arial',
    'Times New Roman', 
    
    # Generic fallbacks (always work)
    'sans-serif',
    'serif',
    'monospace',
    
    # Additional reliable options
    'Helvetica',        # Mac/Linux, falls back to Arial on Windows
    'Times',           # Cross-platform serif
    'Georgia',         # Web-safe serif, good readability
    
    # Office fonts (if available)
    'Calibri',         # Modern, clean sans-serif
    'Cambria',         # Modern serif designed for screen/print
]
charts = []
for font in fonts_to_test:
    chart = alt.Chart().mark_text(fontSize=14, font=font).encode(
        x=alt.value(100), y=alt.value(50), text=alt.value(f'{font}')
    ).properties(width=180, height=80, title=font[:15])
    charts.append(chart)

# Display in grid (4 columns)
alt.vconcat(*[alt.hconcat(*charts[i:i+4]) for i in range(0, len(charts), 4)])

alt.VConcatChart(...)

#### Color Pallets

In [5]:
vega_colors = {
    'blue': '#4c78a8',
    'orange': '#f58518',
    'red': '#e45756',
    'teal': '#72b7b2',
    'green': '#54a24b',
    'yellow': '#eeca3b',
    'purple': '#b279a2',
    'pink': '#ff9da6',
    'brown': '#9d755d',
    'gray': '#bab0ac'
}

In [6]:
matplotlib_colors = {
    'blue': '#1f77b4',
    'orange': '#ff7f0e',
    'green': '#2ca02c',                                                                                                                                                                                      
    'red': '#d62728',
    'purple': '#9467bd',
    'brown': '#8c564b',
    'pink': '#e377c2',
    'gray': '#7f7f7f',
    'olive': '#bcbd22',
    'cyan': '#17becf'
} 

In [7]:
tableau10_colors = {                                                                                                                                                                                         
      'blue': '#4e79a7',                                                                                                                                                                                       
      'orange': '#f28e2b',                                                                                                                                                                                     
      'red': '#e15759',                                                                                                                                                                                        
      'teal': '#76b7b2',                                                                                                                                                                                       
      'green': '#59a14f',                                                                                                                                                                                      
      'yellow': '#edc948',                                                                                                                                                                                     
      'purple': '#b07aa1',                                                                                                                                                                                     
      'pink': '#ff9da7',                                                                                                                                                                                       
      'brown': '#9c755f',                                                                                                                                                                                      
      'gray': '#bab0ac'  # This matches your gray exactly!                                                                                                                                                     
  }                                                          

#### Utilities

In [8]:
def save_chart_alt(chart, title="", fig_path='.', save_as=None, file_format='pdf', 
                   hide=False, scale_factor=2.0):
    """
    Save an Altair chart with flexible saving options and optional display.
    
    Useful for saving charts that have been modified after generation, such as
    adding layers, annotations, or combining multiple charts. Provides the same
    saving logic and parameter structure as generate_bar_graph_alt.
    
    Parameters
    ----------
    chart : altair.Chart
        The Altair chart object to save and optionally display.
        
    title : str, default ""
        Chart title. Used to generate filename when save_as is not provided.
        
    fig_path : str or None, default None
        Directory path to save the chart. 
        Defaults to '.'
        
    save_as : str or None, default None
        File name (excluding extension) to save file.
        If None, uses 'title' or generates timestamp-based filename
        
    file_format : str, default 'png'
        File format for saving: 'png', 'pdf', 'svg'
        
    hide : bool, default False
        If True, chart is saved but not displayed.
        If False, chart is both saved and displayed.
        
    scale_factor : float, default 2.0
        Scale factor for PNG output (higher values = higher resolution).
        Only applies to PNG format.
        
    Returns
    -------
    None
        Function saves file and optionally displays chart.
        
    Examples
    --------
    >>> # Generate base chart
    >>> chart, title = generate_bar_graph_alt(df, y_col='Coverage', 
    ...                                       title='Sample Coverage', 
    ...                                       return_chart=True)
    >>> 
    >>> # Add vertical line
    >>> rule = alt.Chart().mark_rule().encode(x=alt.datum(1.5))
    >>> modified_chart = chart + rule
    >>> 
    >>> # Save with same title
    >>> save_chart_alt(modified_chart, title)
    >>> 
    >>> # Save with custom filename and hide display
    >>> save_chart_alt(modified_chart, title, save_as='coverage_with_line', 
    ...                 hide=True)
    >>> 
    >>> # Save as PDF in custom directory
    >>> save_chart_alt(modified_chart, title, fig_path='analysis/plots',
    ...                 file_format='pdf')
        
    Notes
    -----
    - Uses same file naming logic as generate_bar_graph_alt
    - PNG files are saved with scale_factor for higher resolution
    - Other formats (PDF, SVG) ignore scale_factor parameter
    - If no title or save_as provided, generates timestamp-based filename
    """
    if save_as is None: save_as = title or f"chart_{datetime.now().strftime('%y%m%d_%H%M%S')}"
    save_as = sanitize(save_as)
    #path = fname(fig_path, save_as, file_format)
    path = f'{fig_path}/{save_as}.{file_format}'
    if file_format == 'png': chart.save(path, scale_factor=scale_factor)
    else: chart.save(path)
    if not hide: display(chart)

def sanitize(title: str) -> str:                                                                                                                                                                               
    """Sanitize string by replacing specific chars with underscores."""                                                                                                                                        
    sanitized = re.sub(r'[^a-zA-Z0-9\s]', '_', title)                                                                                                                                                          
    sanitized = re.sub(r'\s+', '_', sanitized)                                                                                                                                                                 
    sanitized = re.sub(r'_+', '_', sanitized)                                                                                                                                                                  
    return sanitized.strip('_')

## Vizard

In [6]:
%load_ext cc_jupyter 


🚀 Claude Code Magic loaded!
Features:
  • Full agentic Claude Code execution
  • Cell-based code approval workflow
  • Real-time message streaming
  • Session state preservation
  • Conversation continuity across cells

Usage:
  %cc <instructions>       # Continue with additional instructions (one-line)
  %%cc <instructions>      # Continue with additional instructions (multi-line)
  %cc_new (or %ccn)        # Start fresh conversation
  %cc --help               # Show available options and usage information

Context management:
  %cc --import <file>       # Add a file to be included in initial conversation messages
  %cc --add-dir <dir>       # Add a directory to Claude's accessible directories
  %cc --mcp-config <file>   # Set path to a .mcp.json file containing MCP server configurations
  %cc --cells-to-load <num> # The number of cells to load into a new conversation (default: all for first %cc, none for %cc_new)

Output:
  %cc --model <name>       # Model to use for Claude Code (de

In [7]:
%cc RESET

🧠 Claude model: claude-sonnet-4-5-20250929
💭 Claude: I'll reset the Vizard state to defaults.
⏺ Bash("rm -f .vizard_state.json && cat > .vizard_state.json << 'EOF'
{
  "ENGINE": "altair",
  "DF": "polars",
  "WIDTH": 600,
  "HEIGHT": 400,
  "FUNCTION": false,
  "IMPORT": false,
  "OUTPUT": "display"
}
EOF
")
💭 Claude: ✓ State reset to defaults
📍 Claude Code Session ID: d31fb261-1684-4d06-b377-346ac363a5dc


In [8]:
%cc --model sonnet

✅ Set model to sonnet. Use %cc_new to pick up the setting.


## Analysis